## Part 3 - Train Model

**Connect to Azure ML workspace**

In [8]:
from azureml.core import Workspace

workspace = Workspace.from_config()

**Retrieve dataset and compute targets prepared in earlier steps**

In [9]:
cifar_dataset = workspace.datasets['CIFAR-10 Dataset']
compute_target = workspace.compute_targets['k80cluster']

**Create experiment**

In [10]:
from azureml.core import Experiment

experiment_name = 'objection-recognition' 
experiment = Experiment(workspace, name=experiment_name)

**Submit training job**

In [17]:
%pycat train/train.py

In [17]:
from azureml.train.estimator import Estimator

estimator = Estimator(source_directory='train',
                      entry_script='train.py',
                      compute_target=compute_target,
                      script_params = {
                          '--data_dir': cifar_dataset.as_named_input('cifardata').as_mount(),
                          '--output_dir': './outputs', 
                          '--num_epochs': 10, 
                          '--batch_size': 16,
                          '--learning_rate': 0.001, 
                          '--momentum': 0.9
                      },
                      use_gpu=True,
                      pip_packages=['torch==1.4.0', 'torchvision==0.5.0', 'azureml-dataprep[fuse,pandas]==1.1.38'])

In [18]:
run = experiment.submit(estimator)

In [19]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

**Register model**

In [20]:
model = run.register_model(model_name='cifar-classifier', model_path='outputs')